# Plot Result

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import pickle
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
def cv_result(fileName, e):
    import os
    from tqdm import tqdm
    filePath, r = [], [0]*e
    for i in os.walk(fileName):
        if i != [] and i[-1] != []:
            filePath.extend(i[-1])
            filePath = [fileName+fp for fp in filePath]
    
    for fp in filePath:
        cv = pickle.load(open(fp, 'rb'))
        for i in range(len(cv[-1])):
            r[i] += cv[-1][i]
    return [(i[0], i[-1]/len(filePath)) for i in enumerate(r, 1)]


def plot_ctl(file_path, num_epochs=200, show=False, save_path=None):
    """
    Plot f1-score for causal triplet labeling
    """
    f1 = [i[-1] for i in cv_result(file_path, num_epochs)]
    max_f1, epoch = max(f1), list(f1).index(max(f1))+1
    epochs = list(range(1, num_epochs+1))
    plt.figure(figsize=(10, 8))
    plt.plot(epochs, f1, label="F1-Score", color="green", linewidth=1)
    plt.xlabel("Epochs")
    plt.ylabel("Causal Triplet Labeling")
    plt.xticks([i for i in range(1, len(f1), 14)])
    plt.grid(True)
    plt.legend()
    plt.annotate('max: %.3f epoch: %3d' % (max_f1, epoch), xy=(epoch, max_f1-0.01),
                 xytext=(epoch-40, max_f1-0.05), arrowprops=dict(facecolor='black'))

    if save_path != None:
        plt.savefig(save_path, dpi=666)


def plot_loss(file_path, num_epochs=200, show=False, save_path=None):
    """
    Plot Loss
    """
    log = np.load(open(file_path, 'rb'))
    plt.figure(figsize=(10, 8))
    loss, val_loss = sum([i['loss'] for i in log], []), sum(
        [i['val_loss'] for i in log], [])
    min_loss, min_val_loss = min(loss), min(val_loss)
    loss_epoch, val_loss_epoch = list(loss).index(
        min(loss))+1, list(val_loss).index(min(val_loss))+1
    epochs = list(range(1, num_epochs+1))
    plt.plot(epochs, val_loss, label="val_loss", color="red", linewidth=1)
    plt.plot(epochs, loss, label="train_loss", color="green", linewidth=1)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.xticks([i for i in range(1, len(loss), 16)])
    plt.grid(True)
    plt.annotate('min: %.3f epoch: %3d' % (min_loss, loss_epoch), xy=(loss_epoch, min_loss+0.1),
                 xytext=(loss_epoch-40, min_loss+2), arrowprops=dict(facecolor='black', shrink=0.05))
    plt.annotate('min: %.3f epoch: %3d' % (min_val_loss, val_loss_epoch), xy=(val_loss_epoch, min_val_loss+0.1),
                 xytext=(val_loss_epoch-40, min_val_loss+2), arrowprops=dict(facecolor='black', shrink=0.05))
    plt.legend()

    if save_path != None:
        plt.savefig(save_path, dpi=666)

# plot_ctl('yout path to cv log')

# Evaluate

## Build Model

In [7]:
import keras
import numpy as np
import os
import random as rn
import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
from keras import backend as K
import pickle
from keras.utils import to_categorical
import h5py
from keras.layers import *
import math
from MHSA import MultiHeadSelfAttention
from ChainCRF import ChainCRF
from keras.models import Model
from keras import optimizers
from keras.callbacks import*
from tag2triplet import*
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse

In [49]:
# -*- coding: utf-8 -*-

'''
Author: Zhaoning Li
'''

import keras
import numpy as np
import os
import random as rn
import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
from keras import backend as K
import pickle
from keras.utils import to_categorical
import h5py
from keras.layers import *
import math
from MHSA import MultiHeadSelfAttention
from ChainCRF import ChainCRF
from keras.models import Model
from keras import optimizers
from keras.callbacks import*
from tag2triplet import*
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse


MAX_WLEN = 58
MAX_CLEN = 23
VOCAB_SIZE = 15539
CHAR_SIZE = 69
EXTVEC_DIM = 300
FLAIR_DIM = 4096
CHAR_DIM = 30
NUM_CHAR_CNN_FILTER = 30
CHAR_CNN_KERNEL_SIZE = 3
CHAR_LSTM_SIZE = 25
NUM_ID_CNN_FILTER = 300
ID_CNN_KERNEL_SIZE = 3
DILATION_RATE = (1, 2, 4, 1)
NUM_CLASS = 7


class MaskConv1D(Conv1D):
    def __init__(self, **kwargs):
        super(MaskConv1D, self).__init__(**kwargs)
        self.supports_masking = True

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            inputs *= K.expand_dims(mask, axis=-1)
        return super(MaskConv1D, self).call(inputs)


class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, x, x_flair, x_char, y, batch_size, classifier, pred=False):
        self.list_IDs = list_IDs
        self.x = x
        self.x_flair = x_flair
        self.x_char = x_char
        self.y = y
        self.batch_size = batch_size
        self.classifier = classifier
        self.pred = pred

    def __len__(self):
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        list_IDs_temp = self.list_IDs[index *
                                      self.batch_size:(index+1)*self.batch_size]
        return self.__data_generation(list_IDs_temp)

    def __data_generation(self, list_IDs_temp):
        x = []
        if self.pred:
            maxlen = MAX_WLEN
        else:
            maxlen = max([len(np.where(self.x[ID] != 0)[0])
                          for ID in list_IDs_temp])
        x.append(np.zeros((self.batch_size, maxlen)))

        for i, ID in enumerate(list_IDs_temp):
            x[0][i] = self.x[ID][:maxlen]

        if self.x_flair != None:
            x_flair = np.zeros((self.batch_size, maxlen, FLAIR_DIM))
            for i, ID in enumerate(list_IDs_temp):
                x_flair[i] = self.x_flair[ID][:maxlen]
            x.append(x_flair)

        if self.x_char != None:
            if self.pred:
                maxlen_c = MAX_CLEN
            else:
                maxlen_c = max([len(np.where(self.x[ID][_] != 0)[0]) for _ in range(maxlen) for ID in list_IDs_temp])
            x_char = np.zeros((self.batch_size, maxlen, maxlen_c))
            for i, ID in enumerate(list_IDs_temp):
                x_char[i] = self.x_char[ID][:maxlen][:, :maxlen_c]
            x.append(x_char)

        if self.pred:
            return x

        y = np.zeros((self.batch_size, maxlen, 1))
        for i, ID in enumerate(list_IDs_temp):
            y[i] = self.y[ID][:maxlen]

        return x, y


class Data:
    def __init__(self, args):
        self.word2index, self.index2word = pickle.load(
            open(args.file_path+'index/index_w.pkl', 'rb'))
        self.embedding = np.load(
            open(args.file_path+'embedding/extvec_embedding.npy', 'rb'))

        with h5py.File(args.file_path+'test/test.h5', 'r') as fh:
            self.xTest = fh['xTest'][:]
            self.yTest = fh['yTest'][:] 

        self.test_flair, self.test_char = None, None    
        if args.use_flair:
            h5f = h5py.File(args.file_path+'embedding/flair.h5', 'r')
            self.test_flair = h5f['xTest_flair']
        if args.char_emb != None:
            h5f_te = h5py.File(args.file_path+'test/test.h5', 'r')
            self.test_char = h5f_te['xTest_c']
            
        y_true = self.yTest.reshape(self.yTest.shape[0], MAX_WLEN)
        self.y_true_idx = [final_result(y_true[i], [
            self.index2word[w] for w in self.xTest[i] if w != 0]) for i in range(len(y_true))]
        self.ap = sum([len(i) for i in self.y_true_idx if i != 0])


class Evaluate(Callback):
    def __init__(self,
                 args,
                 data,
                 x,
                 x_g,
                 y_true_idx,
                 ap,
                 save_path=None):
        self.pre = []
        self.rec = []
        self.f1 = []
        self.best_f1 = 0.
        self.x = x
        self.x_g = x_g
        self.y_true_idx = y_true_idx
        self.ap = ap
        #self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        if args.objective == 'cv':
            y_pred = np.argmax(self.model.predict_generator(self.x_g), axis=-1)[:len(self.x)]
            y_pred_idx = [final_result(
                y_pred[i], [data.index2word[w] for w in self.x[i] if w != 0]) for i in range(len(y_pred))]
            pp = sum([len(i) for i in y_pred_idx if i != 0])
            tp = 0
            for i in range(len(self.y_true_idx)):
                if self.y_true_idx[i] != 0 and y_pred_idx[i] != 0:
                    for m in self.y_true_idx[i]:
                        y_true_cause = [data.index2word[self.x[i][idx]]
                                        for idx in m[0]]
                        y_true_effect = [data.index2word[self.x[i][idx]]
                                         for idx in m[-1]]
                        for n in y_pred_idx[i]:
                            y_pred_cause = [data.index2word[self.x[i][idx]]
                                            for idx in n[0] if self.x[i][idx] != 0]
                            y_pred_effect = [data.index2word[self.x[i][idx]]
                                             for idx in n[-1] if self.x[i][idx] != 0]
                            if y_true_cause == y_pred_cause and y_true_effect == y_pred_effect:
                                tp += 1

            pre = tp / float(pp) if pp != 0 else 0
            rec = tp / float(self.ap) if self.ap != 0 else 0
            f1 = 2 * pre * rec / float(pre + rec) if (pre + rec) != 0 else 0
            self.pre.append(pre)
            self.rec.append(rec)
            self.f1.append(f1)
            if f1 > self.best_f1:
                self.best_f1 = f1
            print(' - val_precision: %.4f - val_recall: %.4f - val_f1_score: %.4f - best_f1_score: %.4f' %
                  (pre, rec, f1, self.best_f1))
            if epoch + 1 == args.num_epochs:
                isExists = os.path.exists(self.save_path+'/cv/'+str(args.seed))
                if not isExists:
                    os.makedirs(self.save_path+'/cv/'+str(args.seed))
                with open(self.save_path+'/cv/'+str(args.seed)+'/'+str(args.k_fold)+'.pkl', 'wb') as fp:
                    pickle.dump((self.pre, self.rec, self.f1), fp, -1)
        if args.objective == 'test':
            if epoch + 1 > 0:
                isExists = os.path.exists(self.save_path+'/test')
                if not isExists:
                    os.makedirs(self.save_path+'/test')
                self.model.save(filepath=self.save_path+'/test/'+str(args.seed) + '_{' +
                                str(epoch + 1) + '}' + '.hdf5')


class CausalityExtractor:
    def __init__(self, args):
        self.reproducibility()
        self.kernel_initializer = keras.initializers.glorot_uniform(
            seed=args.seed)
        self.recurrent_initializer = keras.initializers.Orthogonal(
            seed=args.seed)
        self.lr = args.learning_rate
        self.save_path = 'logs/FLAIR-'+str(args.use_flair)+'_CHAR-'+str(args.char_emb) + \
                         '_'+args.backbone.upper()+'_1-512' + \
            '_MHSA-'+str(args.use_att)+'_' + args.classifier.upper()

    def reproducibility(self):
        """
        Ensure that the model can obtain reproducible results
        """
        os.environ['PYTHONHASHSEED'] = str(args.seed)
        os.environ['CUDA_VISIBLE_DEVICES'] = str(args.cuda_devices)
        np.random.seed(args.seed)
        rn.seed(args.seed)
        ##################configproto
        session_conf = tf.compat.v1.ConfigProto(
            device_count={'CPU': args.cpu_core},
            intra_op_parallelism_threads=args.cpu_core,
            inter_op_parallelism_threads=args.cpu_core,
            gpu_options=tf.compat.v1.GPUOptions(allow_growth=True,
                                     #per_process_gpu_memory_fraction=0.7
                                     ),
            allow_soft_placement=True)
        tf.compat.v1.set_random_seed(args.seed)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
        K.set_session(sess)

    def conv_block(self, x, dilation_rate=1, use_dropout=True, name='1'):
        '''
        Utility function to apply conv.
        '''
        x = MaskConv1D(filters=NUM_ID_CNN_FILTER,
                       kernel_size=ID_CNN_KERNEL_SIZE,
                       padding='same',
                       dilation_rate=dilation_rate,
                       kernel_initializer=self.kernel_initializer,
                       name='CONV-'+name)(x)
        x = Activation('relu', name='RELU-'+name)(x)
        if use_dropout:
            x = Dropout(args.dropout_rate, seed=args.seed,
                        name='DROPOUT-'+name)(x)
        return x

    def slm(self, data):
        """
        Returns Sequence Labeling Model.
        """
        seq = Input(shape=(None,), name='INPUT')
        emb = Embedding(VOCAB_SIZE,
                        EXTVEC_DIM,
                        weights=[data.embedding],
                        mask_zero=True,
                        trainable=False, name='WE')(seq)
        input_node = [seq]

        if args.use_flair:
            flair = Input(shape=(None, FLAIR_DIM), name='FLAIR')
            emb = concatenate([emb, flair], axis=-1, name='EMB_FLAIR')
            input_node.append(flair)

        if args.char_emb != None:
            char_embedding = []
            for _ in range(CHAR_SIZE):
                scale = math.sqrt(3.0 / CHAR_DIM)
                char_embedding.append(
                    np.random.uniform(-scale, scale, CHAR_DIM))
            char_embedding[0] = np.zeros(CHAR_DIM)
            char_embedding = np.asarray(char_embedding)

            char_seq = Input(shape=(None, None), name='CHAR_INPUT')
            char_emb = TimeDistributed(
                Embedding(CHAR_SIZE,
                          CHAR_DIM,
                          weights=[char_embedding],
                          mask_zero=True,
                          trainable=True), name='CHAR_EMB')(char_seq)
            
            if args.char_emb == 'lstm':
                char_emb = TimeDistributed(Bidirectional(LSTM(CHAR_LSTM_SIZE,
                                                              kernel_initializer=self.kernel_initializer,
                                                              recurrent_initializer=self.recurrent_initializer,
                                                              implementation=2,
                                                              return_sequences=False)), name="CHAR_BiLSTM")(char_emb)
                
            if args.char_emb == 'cnn':
                char_emb = TimeDistributed(MaskConv1D(filters=NUM_CHAR_CNN_FILTER,
                                                      kernel_size=CHAR_CNN_KERNEL_SIZE,
                                                      padding='same',
                                                      kernel_initializer=self.kernel_initializer), name="CHAR_CNN")(char_emb)
                char_emb = TimeDistributed(
                    Lambda(lambda x: K.max(x, axis=1)), name="MAX_POOLING")(char_emb)

            input_node.append(char_seq)
            emb = concatenate([emb, char_emb], axis=-1, name='EMB_CHAR')

        if args.backbone == 'lstm':
            
            dec = Bidirectional(LSTM(args.lstm_size,
                                     kernel_initializer=self.kernel_initializer,
                                     recurrent_initializer=self.recurrent_initializer,
                                     dropout=args.dropout_rate,
                                     recurrent_dropout=args.dropout_rate,
                                     implementation=2,
                                     return_sequences=True),
                                merge_mode='concat', name='BiLSTM-1')(emb)
            
            '''
            enc_bilstm = Bidirectional(LSTM(args.lstm_size,
                                            kernel_initializer=self.kernel_initializer,
                                            recurrent_initializer=self.recurrent_initializer,
                                            dropout=args.dropout_rate,
                                            recurrent_dropout=args.dropout_rate,
                                            implementation=2,
                                            return_sequences=True),
                                       merge_mode='concat', name='BiLSTM-1')(emb)
            dec = Bidirectional(LSTM(args.lstm_size,
                                     kernel_initializer=self.kernel_initializer,
                                     recurrent_initializer=self.recurrent_initializer,
                                     dropout=args.dropout_rate,
                                     recurrent_dropout=args.dropout_rate,
                                     implementation=2,
                                     return_sequences=True),
                                merge_mode='concat', name='BiLSTM-2')(enc_bilstm)
            '''
            if args.use_att:
                mhsa = MultiHeadSelfAttention(
                    head_num=args.nb_head, size_per_head=args.size_per_head, kernel_initializer=self.kernel_initializer, name='MHSA')(dec)
                dec = concatenate(
                    [dec, mhsa], axis=-1, name='CONTEXT')

        if args.backbone == 'cnn':
            conv_1 = self.conv_block(
                emb, dilation_rate=DILATION_RATE[0], name='1')
            conv_2 = self.conv_block(
                conv_1, dilation_rate=DILATION_RATE[1], name='2')
            conv_3 = self.conv_block(
                conv_2, dilation_rate=DILATION_RATE[2], name='3')
            dec = self.conv_block(conv_3, dilation_rate=DILATION_RATE[-1],
                                  use_dropout=False, name='4')

        if args.classifier == 'softmax':
            output = TimeDistributed(Dense(NUM_CLASS, activation='softmax',
                                           kernel_initializer=self.kernel_initializer), name='DENSE')(dec)
            loss_func = 'sparse_categorical_crossentropy'

        if args.classifier == 'crf':
            dense = TimeDistributed(Dense(
                NUM_CLASS, activation=None, kernel_initializer=self.kernel_initializer), name='DENSE')(dec)
            crf = ChainCRF(init=self.kernel_initializer, name='CRF')
            output = crf(dense)
            loss_func = crf.sparse_loss

        optimizer = optimizers.Nadam(learning_rate=self.lr, clipnorm=args.clip_norm)
        model = Model(inputs=input_node, outputs=output)
        model.compile(loss=loss_func, optimizer=optimizer)
        return model

In [59]:
# -*- coding: utf-8 -*-

'''
Author: Zhaoning Li
'''

import keras
import numpy as np
import os
import random as rn
import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
from keras import backend as K
import pickle
from keras.utils import to_categorical
import h5py
from keras.layers import *
import math
from MHSA import MultiHeadSelfAttention
from ChainCRF import ChainCRF
from keras.models import Model
from keras import optimizers
from keras.callbacks import*
from tag2triplet import*
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse


MAX_WLEN = 58
MAX_CLEN = 23
VOCAB_SIZE = 15539
CHAR_SIZE = 69
EXTVEC_DIM = 300
FLAIR_DIM = 4096
CHAR_DIM = 30
NUM_CHAR_CNN_FILTER = 30
CHAR_CNN_KERNEL_SIZE = 3
CHAR_LSTM_SIZE = 25
NUM_ID_CNN_FILTER = 300
ID_CNN_KERNEL_SIZE = 3
DILATION_RATE = (1, 2, 4, 1)
NUM_CLASS = 7


class MaskConv1D(Conv1D):
    def __init__(self, **kwargs):
        super(MaskConv1D, self).__init__(**kwargs)
        self.supports_masking = True

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            inputs *= K.expand_dims(mask, axis=-1)
        return super(MaskConv1D, self).call(inputs)


class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, x, x_flair, x_char, y, batch_size, classifier, pred=False):
        self.list_IDs = list_IDs
        self.x = x
        self.x_flair = x_flair
        self.x_char = x_char
        self.y = y
        self.batch_size = batch_size
        self.classifier = classifier
        self.pred = pred

    def __len__(self):
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        list_IDs_temp = self.list_IDs[index *
                                      self.batch_size:(index+1)*self.batch_size]
        return self.__data_generation(list_IDs_temp)

    def __data_generation(self, list_IDs_temp):
        x = []
        if self.pred:
            maxlen = MAX_WLEN
        else:
            maxlen = max([len(np.where(self.x[ID] != 0)[0])
                          for ID in list_IDs_temp])
        x.append(np.zeros((self.batch_size, maxlen)))

        for i, ID in enumerate(list_IDs_temp):
            x[0][i] = self.x[ID][:maxlen]

        if self.x_flair != None:
            x_flair = np.zeros((self.batch_size, maxlen, FLAIR_DIM))
            for i, ID in enumerate(list_IDs_temp):
                x_flair[i] = self.x_flair[ID][:maxlen]
            x.append(x_flair)

        if self.x_char != None:
            if self.pred:
                maxlen_c = MAX_CLEN
            else:
                maxlen_c = max([len(np.where(self.x[ID][_] != 0)[0]) for _ in range(maxlen) for ID in list_IDs_temp])
            x_char = np.zeros((self.batch_size, maxlen, maxlen_c))
            for i, ID in enumerate(list_IDs_temp):
                x_char[i] = self.x_char[ID][:maxlen][:, :maxlen_c]
            x.append(x_char)

        if self.pred:
            return x

        y = np.zeros((self.batch_size, maxlen, 1))
        for i, ID in enumerate(list_IDs_temp):
            y[i] = self.y[ID][:maxlen]

        return x, y


class Data:
    def __init__(self, args):
        self.word2index, self.index2word = pickle.load(
            open(args.file_path+'index/index_w.pkl', 'rb'))
        self.embedding = np.load(
            open(args.file_path+'embedding/extvec_embedding.npy', 'rb'))

        with h5py.File(args.file_path+'test/test.h5', 'r') as fh:
            self.xTest = fh['xTest'][:]
            self.yTest = fh['yTest'][:] 

        self.test_flair, self.test_char = None, None    
        if args.use_flair:
            h5f = h5py.File(args.file_path+'embedding/flair.h5', 'r')
            self.test_flair = h5f['xTest_flair']
        if args.char_emb != None:
            h5f_te = h5py.File(args.file_path+'test/test.h5', 'r')
            self.test_char = h5f_te['xTest_c']
            
        y_true = self.yTest.reshape(self.yTest.shape[0], MAX_WLEN)
        self.y_true_idx = [final_result(y_true[i], [
            self.index2word[w] for w in self.xTest[i] if w != 0]) for i in range(len(y_true))]
        self.ap = sum([len(i) for i in self.y_true_idx if i != 0])


class Evaluate(Callback):
    def __init__(self,
                 args,
                 data,
                 x,
                 x_g,
                 y_true_idx,
                 ap,
                 save_path=None):
        self.pre = []
        self.rec = []
        self.f1 = []
        self.best_f1 = 0.
        self.x = x
        self.x_g = x_g
        self.y_true_idx = y_true_idx
        self.ap = ap
        #self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        if args.objective == 'cv':
            y_pred = np.argmax(self.model.predict_generator(self.x_g), axis=-1)[:len(self.x)]
            y_pred_idx = [final_result(
                y_pred[i], [data.index2word[w] for w in self.x[i] if w != 0]) for i in range(len(y_pred))]
            pp = sum([len(i) for i in y_pred_idx if i != 0])
            tp = 0
            for i in range(len(self.y_true_idx)):
                if self.y_true_idx[i] != 0 and y_pred_idx[i] != 0:
                    for m in self.y_true_idx[i]:
                        y_true_cause = [data.index2word[self.x[i][idx]]
                                        for idx in m[0]]
                        y_true_effect = [data.index2word[self.x[i][idx]]
                                         for idx in m[-1]]
                        for n in y_pred_idx[i]:
                            y_pred_cause = [data.index2word[self.x[i][idx]]
                                            for idx in n[0] if self.x[i][idx] != 0]
                            y_pred_effect = [data.index2word[self.x[i][idx]]
                                             for idx in n[-1] if self.x[i][idx] != 0]
                            if y_true_cause == y_pred_cause and y_true_effect == y_pred_effect:
                                tp += 1

            pre = tp / float(pp) if pp != 0 else 0
            rec = tp / float(self.ap) if self.ap != 0 else 0
            f1 = 2 * pre * rec / float(pre + rec) if (pre + rec) != 0 else 0
            self.pre.append(pre)
            self.rec.append(rec)
            self.f1.append(f1)
            if f1 > self.best_f1:
                self.best_f1 = f1
            print(' - val_precision: %.4f - val_recall: %.4f - val_f1_score: %.4f - best_f1_score: %.4f' %
                  (pre, rec, f1, self.best_f1))
            if epoch + 1 == args.num_epochs:
                isExists = os.path.exists(self.save_path+'/cv/'+str(args.seed))
                if not isExists:
                    os.makedirs(self.save_path+'/cv/'+str(args.seed))
                with open(self.save_path+'/cv/'+str(args.seed)+'/'+str(args.k_fold)+'.pkl', 'wb') as fp:
                    pickle.dump((self.pre, self.rec, self.f1), fp, -1)
        if args.objective == 'test':
            if epoch + 1 > 0:
                isExists = os.path.exists(self.save_path+'/test')
                if not isExists:
                    os.makedirs(self.save_path+'/test')
                self.model.save(filepath=self.save_path+'/test/'+str(args.seed) + '_{' +
                                str(epoch + 1) + '}' + '.hdf5')


class CausalityExtractor:
    def __init__(self, args):
        self.reproducibility()
        self.kernel_initializer = keras.initializers.glorot_uniform(
            seed=args.seed)
        self.recurrent_initializer = keras.initializers.Orthogonal(
            seed=args.seed)
        self.lr = args.learning_rate
        self.save_path = 'logs/FLAIR-'+str(args.use_flair)+'_CHAR-'+str(args.char_emb) + \
                         '_'+args.backbone.upper()+'_1-512' + \
            '_MHSA-'+str(args.use_att)+'_' + args.classifier.upper()

    def reproducibility(self):
        """
        Ensure that the model can obtain reproducible results
        """
        os.environ['PYTHONHASHSEED'] = str(args.seed)
        os.environ['CUDA_VISIBLE_DEVICES'] = str(args.cuda_devices)
        np.random.seed(args.seed)
        rn.seed(args.seed)
        ##################configproto
        session_conf = tf.compat.v1.ConfigProto(
            device_count={'CPU': args.cpu_core},
            intra_op_parallelism_threads=args.cpu_core,
            inter_op_parallelism_threads=args.cpu_core,
            gpu_options=tf.compat.v1.GPUOptions(allow_growth=True,
                                     #per_process_gpu_memory_fraction=0.7
                                     ),
            allow_soft_placement=True)
        tf.compat.v1.set_random_seed(args.seed)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
        K.set_session(sess)

    def conv_block(self, x, dilation_rate=1, use_dropout=True, name='1'):
        '''
        Utility function to apply conv.
        '''
        x = MaskConv1D(filters=NUM_ID_CNN_FILTER,
                       kernel_size=ID_CNN_KERNEL_SIZE,
                       padding='same',
                       dilation_rate=dilation_rate,
                       kernel_initializer=self.kernel_initializer,
                       name='CONV-'+name)(x)
        x = Activation('relu', name='RELU-'+name)(x)
        if use_dropout:
            x = Dropout(args.dropout_rate, seed=args.seed,
                        name='DROPOUT-'+name)(x)
        return x

    def slm(self, data):
        """
        Returns Sequence Labeling Model.
        """
        seq = Input(shape=(None,), name='INPUT')
        emb = Embedding(VOCAB_SIZE,
                        EXTVEC_DIM,
                        weights=[data.embedding],
                        mask_zero=True,
                        trainable=False, name='WE')(seq)
        input_node = [seq]

        if args.use_flair:
            flair = Input(shape=(None, FLAIR_DIM), name='FLAIR')
            emb = concatenate([emb, flair], axis=-1, name='EMB_FLAIR')
            input_node.append(flair)

        if args.char_emb != None:
            char_embedding = []
            for _ in range(CHAR_SIZE):
                scale = math.sqrt(3.0 / CHAR_DIM)
                char_embedding.append(
                    np.random.uniform(-scale, scale, CHAR_DIM))
            char_embedding[0] = np.zeros(CHAR_DIM)
            char_embedding = np.asarray(char_embedding)

            char_seq = Input(shape=(None, None), name='CHAR_INPUT')
            char_emb = TimeDistributed(
                Embedding(CHAR_SIZE,
                          CHAR_DIM,
                          weights=[char_embedding],
                          mask_zero=True,
                          trainable=True), name='CHAR_EMB')(char_seq)
            
            if args.char_emb == 'lstm':
                char_emb = TimeDistributed(Bidirectional(LSTM(CHAR_LSTM_SIZE,
                                                              kernel_initializer=self.kernel_initializer,
                                                              recurrent_initializer=self.recurrent_initializer,
                                                              implementation=2,
                                                              return_sequences=False)), name="CHAR_BiLSTM")(char_emb)
                
            if args.char_emb == 'cnn':
                char_emb = TimeDistributed(MaskConv1D(filters=NUM_CHAR_CNN_FILTER,
                                                      kernel_size=CHAR_CNN_KERNEL_SIZE,
                                                      padding='same',
                                                      kernel_initializer=self.kernel_initializer), name="CHAR_CNN")(char_emb)
                char_emb = TimeDistributed(
                    Lambda(lambda x: K.max(x, axis=1)), name="MAX_POOLING")(char_emb)

            input_node.append(char_seq)
            emb = concatenate([emb, char_emb], axis=-1, name='EMB_CHAR')

        if args.backbone == 'lstm':
            
            dec = Bidirectional(LSTM(args.lstm_size,
                                     kernel_initializer=self.kernel_initializer,
                                     recurrent_initializer=self.recurrent_initializer,
                                     dropout=args.dropout_rate,
                                     recurrent_dropout=args.dropout_rate,
                                     implementation=2,
                                     return_sequences=True),
                                merge_mode='concat', name='BiLSTM-1')(emb)
            
            '''
            enc_bilstm = Bidirectional(LSTM(args.lstm_size,
                                            kernel_initializer=self.kernel_initializer,
                                            recurrent_initializer=self.recurrent_initializer,
                                            dropout=args.dropout_rate,
                                            recurrent_dropout=args.dropout_rate,
                                            implementation=2,
                                            return_sequences=True),
                                       merge_mode='concat', name='BiLSTM-1')(emb)
            dec = Bidirectional(LSTM(args.lstm_size,
                                     kernel_initializer=self.kernel_initializer,
                                     recurrent_initializer=self.recurrent_initializer,
                                     dropout=args.dropout_rate,
                                     recurrent_dropout=args.dropout_rate,
                                     implementation=2,
                                     return_sequences=True),
                                merge_mode='concat', name='BiLSTM-2')(enc_bilstm)
            '''
            if args.use_att:
                mhsa = MultiHeadSelfAttention(
                    head_num=args.nb_head, size_per_head=args.size_per_head, kernel_initializer=self.kernel_initializer, name='MHSA')(dec)
                dec = concatenate(
                    [dec, mhsa], axis=-1, name='CONTEXT')

        if args.backbone == 'cnn':
            conv_1 = self.conv_block(
                emb, dilation_rate=DILATION_RATE[0], name='1')
            conv_2 = self.conv_block(
                conv_1, dilation_rate=DILATION_RATE[1], name='2')
            conv_3 = self.conv_block(
                conv_2, dilation_rate=DILATION_RATE[2], name='3')
            dec = self.conv_block(conv_3, dilation_rate=DILATION_RATE[-1],
                                  use_dropout=False, name='4')

        if args.classifier == 'softmax':
            output = TimeDistributed(Dense(NUM_CLASS, activation='softmax',
                                           kernel_initializer=self.kernel_initializer), name='DENSE')(dec)
            loss_func = 'sparse_categorical_crossentropy'

        if args.classifier == 'crf':
            dense = TimeDistributed(Dense(
                NUM_CLASS, activation=None, kernel_initializer=self.kernel_initializer), name='DENSE')(dec)
            crf = ChainCRF(init=self.kernel_initializer, name='CRF')
            output = crf(dense)
            loss_func = crf.sparse_loss

        optimizer = optimizers.Nadam(learning_rate=self.lr, clipnorm=args.clip_norm)
        model = Model(inputs=input_node, outputs=output)
        model.compile(loss=loss_func, optimizer=optimizer)
        return model

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-fp', '--file_path', type=str, default='C:\\Users\\hjbec\\scite\\data\\', help="")
parser.add_argument('-s', '--seed', type=int, default=666, help="")
parser.add_argument('-kf', '--k_fold', type=int, default=0)
parser.add_argument('-cuda', '--cuda_devices', type=int, default=1)
parser.add_argument('-cpu', '--cpu_core', type=int, default=1)
parser.add_argument('-cse', '--use_flair', type=bool, default=True, help="")
parser.add_argument('-chr', '--char_emb', type=str, default="cnn", help="lstm or cnn or None")
parser.add_argument('-b', '--backbone', type=str, default="lstm", help="lstm or cnn")
parser.add_argument('-ls', '--lstm_size', type=int, default=256, help="")
parser.add_argument('-dp', '--dropout_rate', type=float, default=0.5, help="")
parser.add_argument('-att', '--use_att', type=bool, default=True, help="")
parser.add_argument('-nh', '--nb_head', type=int, default=3, help="")
parser.add_argument('-hs', '--size_per_head', type=int, default=8, help="")
parser.add_argument('-lr', '--learning_rate', type=float, default=0.002, help="")
parser.add_argument('-cl', '--classifier', type=str, default="crf", help="softmax or crf")
parser.add_argument('-cn', '--clip_norm', type=float, default=5.0, help="")
parser.add_argument('-n', '--num_epochs', type=int, default=200, help="")
parser.add_argument('-bs', '--batch_size', type=int, default=16, help="")
args = parser.parse_args()
args.file_path = 'C:\\Users\\hjbec\\scite\\data\\'

data = Data(args)
extractor = CausalityExtractor(args)
model = extractor.slm(data)
model.summary()

In [60]:
parser = argparse.ArgumentParser()
parser.add_argument('-fp', '--file_path', type=str, default='C:\\Users\\hjbec\\scite\\data\\', help="")
parser.add_argument('-s', '--seed', type=int, default=666, help="")
parser.add_argument('-kf', '--k_fold', type=int, default=0)
parser.add_argument('-cuda', '--cuda_devices', type=int, default=1)
parser.add_argument('-cpu', '--cpu_core', type=int, default=1)
parser.add_argument('-cse', '--use_flair', type=bool, default=True, help="")
parser.add_argument('-chr', '--char_emb', type=str, default="cnn", help="lstm or cnn or None")
parser.add_argument('-b', '--backbone', type=str, default="lstm", help="lstm or cnn")
parser.add_argument('-ls', '--lstm_size', type=int, default=256, help="")
parser.add_argument('-dp', '--dropout_rate', type=float, default=0.5, help="")
parser.add_argument('-att', '--use_att', type=bool, default=True, help="")
parser.add_argument('-nh', '--nb_head', type=int, default=3, help="")
parser.add_argument('-hs', '--size_per_head', type=int, default=8, help="")
parser.add_argument('-lr', '--learning_rate', type=float, default=0.002, help="")
parser.add_argument('-cl', '--classifier', type=str, default="crf", help="softmax or crf")
parser.add_argument('-cn', '--clip_norm', type=float, default=5.0, help="")
parser.add_argument('-n', '--num_epochs', type=int, default=200, help="")
parser.add_argument('-bs', '--batch_size', type=int, default=16, help="")
args = parser.parse_args()
args.file_path = 'C:\\Users\\hjbec\\scite\\data\\'

data = Data(args)
extractor = CausalityExtractor(args)
model = extractor.slm(data)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 CHAR_INPUT (InputLayer)        [(None, None, None)  0           []                               
                                ]                                                                 
                                                                                                  
 INPUT (InputLayer)             [(None, None)]       0           []                               
                                                                                                  
 CHAR_EMB (TimeDistributed)     (None, None, None,   2070        ['CHAR_INPUT[0][0]']             
                                30)                                                               
                                                                                            

In [51]:
def compute_f1(tp, ap, pp):
    pre = tp / float(pp) if pp != 0 else 0
    rec = tp / float(ap) if ap != 0 else 0
    f1 = 2 * pre * rec / float(pre + rec) if (pre + rec) != 0 else 0
    return pre, rec, f1


def load_pred(path):
    model.load_weights(path)
    test_generator = DataGenerator([i for i in range(len(data.xTest))],
                                    x=data.xTest,
                                    x_flair=data.test_flair,
                                    x_char=data.test_char,
                                    y=data.yTest,
                                    batch_size=args.batch_size,                                                         
                                    classifier=args.classifier,
                                    pred=True)
    return np.argmax(model.predict_generator(test_generator), axis=-1)[:len(data.xTest)]
    

def evaluation_ctl(path=None, show=False):
    """
    Compute precision, recall and f1-score for causal triplet labeling
    """
    if path:
        y_pred = load_pred(path=path) 
    else:
        y_pred = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    
    y_true = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    pp1, pp2, pp3, pp4, pp5, pp6 = 0, 0, 0, 0, 0, 0
    tp1, tp2, tp3, tp4, tp5, tp6 = 0, 0, 0, 0, 0, 0
    ap1, ap2, ap3, ap4, ap5, ap6 = 236, 229, 238, 230, 9, 16   
    for i, yp in enumerate(y_pred):
        yt = y_true[i]
        for j in range(len(y_true[i])):
            if yp[j] == 1:
                pp1 += 1
                if yt[j] == 1:
                    tp1 += 1
            if yp[j] == 2:
                pp2 += 1
                if yt[j] == 2:
                    tp2 += 1
            if yp[j] == 3:
                pp3 += 1
                if yt[j] == 3:
                    tp3 += 1
            if yp[j] == 4:
                pp4 += 1
                if yt[j] == 4:
                    tp4 += 1
            if yp[j] == 5:
                pp5 += 1
                if yt[j] == 5:
                    tp5 += 1
            if yp[j] == 6:
                pp6 += 1
                if yt[j] == 6:
                    tp6 += 1

    #return y_pred
    #pp, tp, sc, se = 0, 0, 0, 0
    
    pp, tp = 0, 0 
    
    #return [final_result(yp, [data.index2word[w] for w in data.xTest[i] if w != 0]) for i, yp in enumerate(y_pred)]
    
    for i, yp in enumerate(y_pred):
        yp_idx = final_result(
            yp, [data.index2word[w] for w in data.xTest[i] if w != 0])
        if yp_idx != 0:
            pp += len(yp_idx)
            if data.y_true_idx[i] != 0:
                y_true_Cause = [[data.index2word[data.xTest[i][idx]]
                                 for idx in m[0]] for m in data.y_true_idx[i]]
                y_true_Effect = [[data.index2word[data.xTest[i][idx]]
                                 for idx in m[-1]] for m in data.y_true_idx[i]]
                y_true_Triplet = [(y_true_Cause[i], y_true_Effect[i])
                                  for i in range(len(y_true_Cause))]
                log_c, log_e = [], []
                for m in data.y_true_idx[i]:
                    y_true_cause = [
                        data.index2word[data.xTest[i][idx]] for idx in m[0]]
                    y_true_effect = [
                        data.index2word[data.xTest[i][idx]] for idx in m[-1]]
                    for n in yp_idx:
                        y_pred_cause = [data.index2word[data.xTest[i][idx]]
                                        for idx in n[0] if data.xTest[i][idx] != 0]
                        y_pred_effect = [data.index2word[data.xTest[i][idx]]
                                             for idx in n[-1] if data.xTest[i][idx] != 0]
                        if y_true_cause == y_pred_cause and y_true_effect == y_pred_effect:
                            tp += 1
                        #if y_true_cause == y_pred_cause and y_true_effect != y_pred_effect and (y_pred_cause, y_pred_effect) not in y_true_Triplet:#+log_c:
                        #if y_true_effect != y_pred_effect and (y_pred_cause, y_pred_effect) not in y_true_Triplet+log_c:
                        #    if (n[0], n[-1]) not in log_c:
                        #        sc += 1
                        #        log_c.append((n[0], n[-1]))
                            #log_c.append((y_pred_cause, y_pred_effect))
                        #if y_true_cause != y_pred_cause and y_true_effect == y_pred_effect and (y_pred_cause, y_pred_effect) not in y_true_Triplet:#+log_e:
                        #if y_true_cause != y_pred_cause and  (y_pred_cause, y_pred_effect) not in y_true_Triplet+log_e:
                        #    if (n[0], n[-1]) not in log_e:
                        #        se += 1
                        #        log_e.append((n[0], n[-1]))
                            #log_e.append((y_pred_cause, y_pred_effect))
                            

    pre, rec, f1 = compute_f1(tp, data.ap, pp)
    #pre1, rec1, f11 = compute_f1(tp1, ap1, pp1)
    #pre2, rec2, f12 = compute_f1(tp2, ap2, pp2)
    #pre3, rec3, f13 = compute_f1(tp3, ap3, pp3)
    #pre4, rec4, f14 = compute_f1(tp4, ap4, pp4)
    #pre5, rec5, f15 = compute_f1(tp5, ap5, pp5)
    #pre6, rec6, f16 = compute_f1(tp6, ap6, pp6)
    preC, recC, f1C = compute_f1(tp1+tp2, ap1+ap2, pp1+pp2)
    preE, recE, f1E = compute_f1(tp3+tp4, ap3+ap4, pp3+pp4)
    preEmb, recEmb, f1Emb = compute_f1(tp5+tp6, ap5+ap6, pp5+pp6)
    #print(tp1, tp2, ap1, ap2, pp1, pp2)
    #scr = sc / float(data.ap)
    #scr = sc / float(pp)
    #ser = se / float(data.ap)
    #ser = se / float(pp)
    
    #print('Triplet: pre: %.4f' % pre, '   rec: %.4f' % rec, '   f1: %.4f' % f1)
    #print('B-C:     pre: %.4f' % pre1, '   rec: %.4f' % rec1, '   f1: %.4f' % f11)
    #print('I-C:     pre: %.4f' % pre2, '   rec: %.4f' % rec2, '   f1: %.4f' % f12)
    #print('B-E:     pre: %.4f' % pre3, '   rec: %.4f' % rec3, '   f1: %.4f' % f13)
    #print('I-E:     pre: %.4f' % pre4, '   rec: %.4f' % rec4, '   f1: %.4f' % f14)
    #print('B-Emb:   pre: %.4f' % pre5, '   rec: %.4f' % rec5, '   f1: %.4f' % f15)
    #print('I-Emb:   pre: %.4f' % pre6, '   rec: %.4f' % rec6, '   f1: %.4f' % f16)
    #print('single-cause: %.4f' % scr, '       single-effect: %.4f' % ser)
    #return pre, rec, f1, pre1, rec1, f11, pre2, rec2, f12, pre3, rec3, f13, pre4, rec4, f14, pre5, rec5, f15, pre6, rec6, f16, scr, ser  
    if show:
        print('Triplet: pre: %.4f' % pre, '   rec: %.4f' % rec, '   f1: %.4f' % f1)
        print('C:       pre: %.4f' % preC, '   rec: %.4f' % recC, '   f1: %.4f' % f1C)
        print('E:       pre: %.4f' % preE, '   rec: %.4f' % recE, '   f1: %.4f' % f1E)
        print('Emb:     pre: %.4f' % preEmb, '   rec: %.4f' % recEmb, '   f1: %.4f' % f1Emb)
        #print('single-cause: %.4f' % scr, '       single-effect: %.4f' % ser)
    return pre, rec, f1, preC, recC, f1C, preE, recE, f1E, preEmb, recEmb, f1Emb


def evaluation_ctl_distance(path=None, show=False, mode="1"):
    """
    Compute precision, recall and f1-score for causal triplet labeling
    """
    if path:
        y_pred = load_pred(path=path) 
    else:
        y_pred = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    
    y_true = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    
    ap, pp, tp = 0, 0, 0
    
    
    if mode == "1":
        minD = 0
        maxD = 5
    if mode == "2":
        minD = 6
        maxD = 10
    if mode == "3":
        minD = 11
        maxD = MAX_WLEN
    if mode == "4":
        minD = 0
        maxD = MAX_WLEN
    '''
    if mode == "1":
        minD = 0
        maxD = 4
    if mode == "2":
        minD = 5
        maxD = 6
    if mode == "3":
        minD = 7
        maxD = 9
    if mode == "4":
        minD = 10
        maxD = MAX_WLEN
    if mode == "5":
        minD = 0
        maxD = MAX_WLEN
    '''
    for i, yp in enumerate(y_pred):
        if data.y_true_idx[i] != 0:
            ap += len([k for k in [max(sum(j, []))-min(sum(j, [])) for j in data.y_true_idx[i]] if k <= maxD and k >= minD])
        yp_idx = final_result(
            yp, [data.index2word[w] for w in data.xTest[i] if w != 0])
        if yp_idx != 0:
            pp += len([k for k in [max(sum(j, []))-min(sum(j, [])) for j in yp_idx] if k <= maxD and k >= minD])
            if data.y_true_idx[i] != 0:
                for m in data.y_true_idx[i]:
                    if max(sum(m, []))-min(sum(m, [])) <= maxD and max(sum(m, []))-min(sum(m, [])) >= minD:
                        y_true_cause = [
                            data.index2word[data.xTest[i][idx]] for idx in m[0]]
                        y_true_effect = [
                            data.index2word[data.xTest[i][idx]] for idx in m[-1]]
                        for n in yp_idx:
                            if max(sum(n, []))-min(sum(n, [])) <= maxD and max(sum(n, []))-min(sum(n, [])) >= minD:
                                y_pred_cause = [data.index2word[data.xTest[i][idx]]
                                                for idx in n[0] if data.xTest[i][idx] != 0]
                                y_pred_effect = [data.index2word[data.xTest[i][idx]]
                                                     for idx in n[-1] if data.xTest[i][idx] != 0]
                                if y_true_cause == y_pred_cause and y_true_effect == y_pred_effect:
                                    tp += 1
    pre, rec, f1 = compute_f1(tp, ap, pp)
    if show:
        print('Triplet: pre: %.4f' % pre, '   rec: %.4f' % rec, '   f1: %.4f' % f1)
    return tp, ap, pp
    return pre, rec, f1


def evaluation_ctl_ea(path=None):
    if path:
        y_pred = load_pred(path=path) 
    else:
        y_pred = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    
    y_true = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    c2e, c2emb, c2o = 0, 0, 0
    e2c, e2emb, e2o = 0, 0, 0
    emb2c, emb2e, emb2o = 0, 0, 0
    o2c, o2e, o2emb = 0, 0, 0
    for i, yp in enumerate(y_pred):
        yt = y_true[i]
        for j in range(len(y_true[i])):
            if yt[j] in [1, 2]:
                if yp[j] in [3, 4]:
                    c2e += 1
                if yp[j] in [5, 6]:
                    c2emb += 1
                if yp[j] == 0:
                    c2o += 1
            if yt[j] in [3, 4]:
                if yp[j] in [1, 2]:
                    e2c += 1
                if yp[j] in [5, 6]:
                    e2emb += 1
                if yp[j] == 0:
                    e2o += 1
            if yt[j] in [5, 6]:
                if yp[j] in [1, 2]:
                    emb2c += 1
                if yp[j] in [3, 4]:
                    emb2e += 1
                if yp[j] == 0:
                    emb2o += 1
            if yt[j] == 0:
                if yp[j] in [1, 2]:
                    o2c += 1
                if yp[j] in [3, 4]:
                    o2e += 1
                if yp[j] in [5, 6]:
                    o2emb += 1       
    return c2e, c2emb, c2o, e2c, e2emb, e2o, emb2c, emb2e, emb2o, o2c, o2e, o2emb


def sprint_distance(fileName, mode="1"):
    pre, rec, f1 = [], [], []
    fileName = [fileName+i for i in sum([i[-1] for i in os.walk(fileName)], [])]
    for fn in fileName:
        _ = evaluation_ctl(fn, mode=mode)
        pre.append(_[0]), rec.append(_[1]), f1.append(_[2]);
    print('Triplet:   pre: %.4f±%.4f' % (np.mean(pre), np.std(pre)), '   rec: %.4f±%.4f' % (np.mean(rec), np.std(rec)), '   f1: %.4f±%.4f' % (np.mean(f1), np.std(f1)))
    
    
def sprint_ea(fileName):
    c2e, c2emb, c2o, e2c, e2emb, e2o, emb2c, emb2e, emb2o, o2c, o2e, o2emb = [], [], [], [], [], [], [], [], [], [], [], []
    fileName = [fileName+i for i in sum([i[-1] for i in os.walk(fileName)], [])]
    for fn in fileName:
        _ = evaluation_ctl_ea(fn)
        c2e.append(_[0]), c2emb.append(_[1]), c2o.append(_[2]);
        e2c.append(_[3]), e2emb.append(_[4]), e2o.append(_[5]);
        emb2c.append(_[6]), emb2e.append(_[7]), emb2o.append(_[8]);
        o2c.append(_[9]), o2e.append(_[10]), o2emb.append(_[11]);
        #print('      C      E      Emb     O')
        #print('C     -      %.2f   %.2f    %.2f' % (np.mean(e2c[-1]), np.mean(emb2c[-1]), np.mean(o2c[-1])))
        #print('E     %.2f   -      %.2f    %.2f' % (np.mean(c2e[-1]), np.mean(emb2e[-1]), np.mean(o2e[-1])))
        #print('Emb   %.2f   %.2f    -      %.2f' % (np.mean(c2emb[-1]), np.mean(e2emb[-1]), np.mean(o2emb[-1])))
        #print('O     %.2f   %.2f   %.2f    -' % (np.mean(c2o[-1]), np.mean(e2o[-1]), np.mean(emb2o[-1])))
    print('      C      E      Emb     O')
    print('C     -      %.2f   %.2f    %.2f' % (np.mean(e2c), np.mean(emb2c), np.mean(o2c)))
    print('E     %.2f   -      %.2f    %.2f' % (np.mean(c2e), np.mean(emb2e), np.mean(o2e)))
    print('Emb   %.2f   %.2f    -      %.2f' % (np.mean(c2emb), np.mean(e2emb), np.mean(o2emb)))
    print('O     %.2f   %.2f   %.2f    -' % (np.mean(c2o), np.mean(e2o), np.mean(emb2o)))
    return c2e, c2emb, c2o, e2c, e2emb, e2o, emb2c, emb2e, emb2o, o2c, o2e, o2emb
    
    
def mhsa_analysis(fileName):
    #for m in [1, 2, 3, 4, 5]:
    for m in [1, 2, 3, 4]:
        sprint_distance(fileName, mode=str(m))
    
    
def sprint(fileName, tag=False):
    pre, rec, f1 = [], [], []
    preC, recC, f1C = [], [], [] 
    preE, recE, f1E = [], [], [] 
    preEmb, recEmb, f1Emb = [], [], []
    #scr, ser = [], []
    fileName = [fileName+i for i in sum([i[-1] for i in os.walk(fileName)], [])]
    for fn in fileName:
        _ = evaluation_ctl(fn)
        pre.append(_[0]), rec.append(_[1]), f1.append(_[2]);
        preC.append(_[3]), recC.append(_[4]), f1C.append(_[5]);
        preE.append(_[6]), recE.append(_[7]), f1E.append(_[8]); 
        preEmb.append(_[9]), recEmb.append(_[10]), f1Emb.append(_[11]); 
        #scr.append(_[-2]), ser.append(_[-1]); 
    print('Triplet:   pre: %.4f±%.4f' % (np.mean(pre), np.std(pre)), '   rec: %.4f±%.4f' % (np.mean(rec), np.std(rec)), '   f1: %.4f±%.4f' % (np.mean(f1), np.std(f1)))
    if tag:
        print('C:         pre: %.4f±%.4f' % (np.mean(preC), np.std(preC)), '   rec: %.4f±%.4f' % (np.mean(recC), np.std(recC)), '   f1: %.4f±%.4f' % (np.mean(f1C), np.std(f1C)))
        print('E:         pre: %.4f±%.4f' % (np.mean(preE), np.std(preE)), '   rec: %.4f±%.4f' % (np.mean(recE), np.std(recE)), '   f1: %.4f±%.4f' % (np.mean(f1E), np.std(f1E)))
        print('Emb:       pre: %.4f±%.4f' % (np.mean(preEmb), np.std(preEmb)), '   rec: %.4f±%.4f' % (np.mean(recEmb), np.std(recEmb)), '   f1: %.4f±%.4f' % (np.mean(f1Emb), np.std(f1Emb)))
    #print('single-cause:   %.4f±%.4f' % (np.mean(scr), np.std(scr)), '              single-effect: %.4f±%.4f' % (np.mean(ser), np.std(ser)))
    

def predict(path=None):
    """
    Print predict causal triplets
    """
    if path:
        y_pred = load_pred(path=path) 
    else:
        y_pred = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)
    
    y_true = data.yTest.reshape(data.yTest.shape[0], MAX_WLEN)

    flag = 1
    for i, yp in enumerate(y_pred):
        yp_idx = final_result(
            yp, [data.index2word[w] for w in data.xTest[i] if w != 0])
        if data.y_true_idx[i] != 0 and 5 not in y_true[i]:
            print('------------------')
            print('Sentence-%.3d' % flag,
                  ' '.join([data.index2word[w] for w in data.xTest[i] if w != 0]))
            y_true_Cause = [[data.index2word[data.xTest[i][idx]]
                             for idx in m[0]] for m in data.y_true_idx[i]]
            y_true_Effect = [[data.index2word[data.xTest[i][idx]]
                               for idx in m[-1]] for m in data.y_true_idx[i]]
            print([(y_true_Cause[i], y_true_Effect[i])
                   for i in range(len(y_true_Cause))])
            if yp_idx != 0:
                y_pred_Cause = [[data.index2word[data.xTest[i][idx]]
                                 for idx in n[0] if data.xTest[i][idx] != 0] for n in yp_idx]
                y_pred_Effect = [[data.index2word[data.xTest[i][idx]]
                                      for idx in n[-1] if data.xTest[i][idx] != 0] for n in yp_idx]
                print([(y_pred_Cause[i], y_pred_Effect[i])
                       for i in range(len(y_pred_Cause))])
            else:
                print([])
            flag += 1

## Main Results

### IDCNN-Softmax

In [28]:
sprint(None)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [52]:
sprint("C:\\Users\\hjbec\\scite\\logs\\logs\\FLAIR-False_CHAR-None_CNN_MHSA-False_SOFTMAX\\test\\")

ValueError: Layer count mismatch when loading weights from file. Model expected 7 layers, found 6 saved layers.

### IDCNN-CRF

In [ ]:
sprint('your path to /FLAIR-False_CHAR-None_CNN_MHSA-False_CRF/test/')

Triplet:   pre: 0.7442±0.0225    rec: 0.7142±0.0122    f1: 0.7288±0.0160


In [33]:
sprint("C:\\Users\\hjbec\\scite\\logs\\logs\\FLAIR-False_CHAR-None_LSTM_MHSA-False_CRF\\test\\")

Triplet:   pre: 1.0000±0.0000    rec: 1.0000±0.0000    f1: 1.0000±0.0000


### BiLSTM-Softmax

In [ ]:
sprint('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-False_SOFTMAX/test/')

Triplet:   pre: 0.7744±0.0183    rec: 0.7622±0.0114    f1: 0.7682±0.0138


In [34]:
sprint('C:\\Users\\hjbec\\scite\\logs\\logs\\FLAIR-False_CHAR-None_LSTM_MHSA-False_SOFTMAX\\test\\')

Triplet:   pre: 1.0000±0.0000    rec: 1.0000±0.0000    f1: 1.0000±0.0000


### CLSTM-BiLSTM-CRF

In [ ]:
sprint('your path to /FLAIR-False_CHAR-lstm_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8144±0.0284    rec: 0.7412±0.0073    f1: 0.7757±0.0107


### CCNN-BiLSTM-CRF

In [ ]:
sprint('your path to /FLAIR-False_CHAR-cnn_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8069±0.0199    rec: 0.7520±0.0227    f1: 0.7780±0.0075


### BiLSTM-CRF

In [ ]:
sprint('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-False_CRF/test/', tag=True)

Triplet:   pre: 0.7837±0.0061    rec: 0.7932±0.0087    f1: 0.7884±0.0072
C:         pre: 0.8810±0.0070    rec: 0.8628±0.0094    f1: 0.8718±0.0076
E:         pre: 0.8928±0.0037    rec: 0.8897±0.0040    f1: 0.8913±0.0031
Emb:       pre: 0.4343±0.1899    rec: 0.0960±0.0320    f1: 0.1567±0.0551


### BERT-BiLSTM-CRF

In [ ]:
sprint('your path to /logs/BERT-True-s_CHAR-None_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8277±0.0058    rec: 0.8209±0.0093    f1: 0.8243±0.0049


### Flair+CLSTM-BiLSTM-CRF

In [ ]:
sprint('your path to /logs/FLAIR-True_CHAR-lstm_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8403±0.0090    rec: 0.8284±0.0125    f1: 0.8343±0.0106


In [35]:
sprint("C:\\Users\\hjbec\\scite\\logs\\logs\\BERT-True-s_CHAR-None_LSTM_MHSA-False_CRF\\test\\")

Triplet:   pre: 1.0000±0.0000    rec: 1.0000±0.0000    f1: 1.0000±0.0000


In [38]:
sprint("C:\\Users\\hjbec\\scite\\logs\\logs\\FLAIR-True_CHAR-lstm_LSTM_MHSA-False_CRF\\test\\")

Triplet:   pre: 1.0000±0.0000    rec: 1.0000±0.0000    f1: 1.0000±0.0000


### ELMo-BiLSTM-CRF

In [ ]:
sprint('your path to /logs/ELMO-True_CHAR-None_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8361±0.0135    rec: 0.8399±0.0063    f1: 0.8379±0.0092


### Flair-BiLSTM-CRF

In [ ]:
sprint('your path to /logs/FLAIR-True_CHAR-None_LSTM_MHSA-False_CRF/test/', tag=True)

Triplet:   pre: 0.8414±0.0079    rec: 0.8351±0.0141    f1: 0.8382±0.0092
C:         pre: 0.8995±0.0091    rec: 0.8843±0.0142    f1: 0.8917±0.0073
E:         pre: 0.9294±0.0062    rec: 0.8885±0.0075    f1: 0.9084±0.0030
Emb:       pre: 0.8556±0.1975    rec: 0.1360±0.0784    f1: 0.2197±0.1027


### SCITE

In [ ]:
sprint('your path to /logs/FLAIR-True_CHAR-cnn_LSTM_MHSA-True_CRF/test/', tag=True)

Triplet:   pre: 0.8333±0.0042    rec: 0.8581±0.0021    f1: 0.8455±0.0028
C:         pre: 0.8999±0.0079    rec: 0.8998±0.0044    f1: 0.8998±0.0018
E:         pre: 0.9272±0.0073    rec: 0.9021±0.0090    f1: 0.9144±0.0055
Emb:       pre: 0.8489±0.2165    rec: 0.1920±0.1085    f1: 0.2947±0.1477


In [44]:
sprint("C:\\Users\\hjbec\\scite\\logs\\logs\\FLAIR-True_CHAR-cnn_LSTM_MHSA-False_CRF\\test\\")

ValueError: Layer count mismatch when loading weights from file. Model expected 7 layers, found 6 saved layers.

## Error Analysis

### BiLSTM-CRF

In [ ]:
sprint_ea('your path to /logs/FLAIR-False_CHAR-None_LSTM_MHSA-False_CRF/test/')

      C      E      Emb     O
C     -      0.00   10.00    37.60
E     3.40   -      12.20    30.60
Emb   1.40   1.60    -      0.40
O     52.40   46.20   0.40    -


([0, 5, 5, 2, 5],
 [1, 1, 1, 1, 3],
 [58, 47, 51, 50, 56],
 [0, 0, 0, 0, 0],
 [0, 2, 2, 2, 2],
 [49, 43, 48, 45, 46],
 [10, 10, 10, 10, 10],
 [10, 13, 13, 12, 13],
 [1, 0, 0, 1, 0],
 [39, 32, 37, 40, 40],
 [33, 28, 31, 33, 28],
 [0, 2, 0, 0, 0])

### Flair-BiLSTM-CRF

In [ ]:
sprint_ea('your path to /FLAIR-True_CHAR-None_LSTM_MHSA-False_CRF/test/')

      C      E      Emb     O
C     -      0.00   10.00    30.60
E     0.00   -      11.60    16.40
Emb   0.40   0.80    -      0.00
O     48.00   47.80   0.00    -


([0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0],
 [49, 58, 41, 45, 47],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 4, 0],
 [47, 46, 45, 50, 51],
 [10, 10, 10, 10, 10],
 [13, 8, 13, 11, 13],
 [0, 0, 0, 0, 0],
 [28, 28, 39, 32, 26],
 [13, 19, 20, 12, 18],
 [0, 0, 0, 0, 0])

### SCITE

In [ ]:
sprint_ea('your path to /FLAIR-True_CHAR-cnn_LSTM_MHSA-True_CRF/test/')

      C      E      Emb     O
C     -      0.00   7.80    34.40
E     1.00   -      12.40    16.20
Emb   0.00   1.40    -      0.00
O     41.20   40.80   0.00    -


([0, 0, 2, 0, 3],
 [0, 0, 0, 0, 0],
 [45, 41, 37, 45, 38],
 [0, 0, 0, 0, 0],
 [2, 0, 0, 0, 5],
 [45, 46, 35, 41, 37],
 [6, 10, 10, 3, 10],
 [11, 13, 13, 14, 11],
 [0, 0, 0, 0, 0],
 [32, 37, 36, 34, 33],
 [15, 23, 17, 11, 15],
 [0, 0, 0, 0, 0])

## Ablation Analysis

### Flair-BiLSTM-MHSA-CRF

In [ ]:
sprint('your path to /FLAIR-True_CHAR-None_LSTM_MHSA-True-24_CRF/test/')

Triplet:   pre: 0.8269±0.0114    rec: 0.8615±0.0077    f1: 0.8438±0.0090


### BiLSTM-MHSA-CRF

In [ ]:
sprint('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-True_CRF/test/')

Triplet:   pre: 0.8014±0.0043    rec: 0.8264±0.0070    f1: 0.8137±0.0051


## Analysis of MHSA 

### First Group

#### BiLSTM-CRF

In [ ]:
mhsa_analysis('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8748±0.0059    rec: 0.9160±0.0053    f1: 0.8949±0.0048
Triplet:   pre: 0.7415±0.0111    rec: 0.7484±0.0091    f1: 0.7450±0.0096
Triplet:   pre: 0.6548±0.0090    rec: 0.6122±0.0365    f1: 0.6324±0.0227
Triplet:   pre: 0.7837±0.0061    rec: 0.7932±0.0087    f1: 0.7884±0.0072


#### BiLSTM-MHSA-CRF

In [ ]:
mhsa_analysis('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-True_CRF/test/')

Triplet:   pre: 0.8990±0.0083    rec: 0.9277±0.0114    f1: 0.9131±0.0092
Triplet:   pre: 0.7587±0.0087    rec: 0.7906±0.0134    f1: 0.7743±0.0073
Triplet:   pre: 0.6747±0.0503    rec: 0.6735±0.0428    f1: 0.6737±0.0436
Triplet:   pre: 0.8014±0.0043    rec: 0.8264±0.0070    f1: 0.8137±0.0051


### Second Group

#### Flair-BiLSTM-CRF

In [ ]:
mhsa_analysis('your path to /FLAIR-True_CHAR-None_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.8970±0.0073    rec: 0.9210±0.0041    f1: 0.9088±0.0034
Triplet:   pre: 0.8272±0.0116    rec: 0.8156±0.0255    f1: 0.8212±0.0171
Triplet:   pre: 0.7236±0.0257    rec: 0.6694±0.0153    f1: 0.6950±0.0114
Triplet:   pre: 0.8414±0.0079    rec: 0.8351±0.0141    f1: 0.8382±0.0092


#### Flair-BiLSTM-MHSA-CRF

In [ ]:
mhsa_analysis('your path to /FLAIR-True_CHAR-None_LSTM_MHSA-True-24_CRF/test/')

Triplet:   pre: 0.8981±0.0156    rec: 0.9328±0.0159    f1: 0.9151±0.0153
Triplet:   pre: 0.8055±0.0193    rec: 0.8453±0.0058    f1: 0.8248±0.0103
Triplet:   pre: 0.7105±0.0160    rec: 0.7306±0.0200    f1: 0.7203±0.0152
Triplet:   pre: 0.8269±0.0114    rec: 0.8615±0.0077    f1: 0.8438±0.0090


### Third Group

#### Flair+CCNN-BiLSTM-CRF

In [ ]:
mhsa_analysis('your path to /FLAIR-True_CHAR-cnn_LSTM_MHSA-False_CRF/test/')

Triplet:   pre: 0.9067±0.0056    rec: 0.9143±0.0034    f1: 0.9105±0.0032
Triplet:   pre: 0.8156±0.0161    rec: 0.8016±0.0245    f1: 0.8084±0.0187
Triplet:   pre: 0.7549±0.0474    rec: 0.6776±0.0327    f1: 0.7126±0.0201
Triplet:   pre: 0.8435±0.0034    rec: 0.8264±0.0143    f1: 0.8348±0.0065


#### SCITE

In [ ]:
mhsa_analysis('your path to /FLAIR-True_CHAR-cnn_LSTM_MHSA-True_CRF/test/')

Triplet:   pre: 0.9039±0.0075    rec: 0.9328±0.0141    f1: 0.9181±0.0088
Triplet:   pre: 0.8134±0.0118    rec: 0.8375±0.0091    f1: 0.8253±0.0096
Triplet:   pre: 0.7060±0.0398    rec: 0.7224±0.0356    f1: 0.7139±0.0353
Triplet:   pre: 0.8333±0.0042    rec: 0.8581±0.0021    f1: 0.8455±0.0028


## Case Study

### 1-175

In [ ]:
predict('your path to /FLAIR-True_CHAR-cnn_LSTM_MHSA-True_CRF/test/6_{96}.hdf5')

------------------
Sentence-001 Cold sores or fever blisters are caused by the herpes simplex virus and are usually relatively easy to identify
[(['the', 'herpes', 'simplex', 'virus'], ['Cold', 'sores']), (['the', 'herpes', 'simplex', 'virus'], ['fever', 'blisters'])]
[(['the', 'herpes', 'simplex', 'virus'], ['Cold', 'sores']), (['the', 'herpes', 'simplex', 'virus'], ['fever', 'blisters'])]
------------------
Sentence-002 Fog , rain , darkness , and or blowing snow lead to disorientation
[(['Fog'], ['disorientation']), (['rain'], ['disorientation']), (['darkness'], ['disorientation']), (['blowing', 'snow'], ['disorientation'])]
[(['Fog'], ['disorientation']), (['rain'], ['disorientation']), (['darkness'], ['disorientation']), (['blowing', 'snow'], ['disorientation'])]
------------------
Sentence-003 Aware of the suffering caused by unmindful drinking , I am committed to cultivate good health , both physical and mental , for myself , my family , and my society
[(['unmindful', 'drinking'

------------------
Sentence-142 In these cases , the disappointment from the purchase is not forgotten over time , but rather accumulates
[(['the', 'purchase'], ['the', 'disappointment'])]
[(['the', 'purchase'], ['the', 'disappointment'])]
------------------
Sentence-143 Four of the entrapments resulted in suffocation : a 7-month-old in Gouverneur , N Y ; a 7-month-old in New Iberia , La ; a 6-month-old in Summersville , W Va ; and a 9-month-old in Bronx , N Y
[(['the', 'entrapments'], ['suffocation'])]
[(['the', 'entrapments'], ['suffocation'])]
------------------
Sentence-144 Muscle fatigue is the number one cause of arm muscle pain
[(['Muscle', 'fatigue'], ['arm', 'muscle', 'pain'])]
[(['Muscle', 'fatigue'], ['arm', 'muscle', 'pain'])]
------------------
Sentence-145 Thus previous scientific estimates had overstated the devastation caused by the asteroid , since topographic and ecologic factors contributing to the result had not been taken into account
[(['the', 'asteroid'], ['the',

In [ ]:
predict('your path to /FLAIR-True_CHAR-None_LSTM_MHSA-False_CRF/test/6_{115}.hdf5')

------------------
Sentence-001 Cold sores or fever blisters are caused by the herpes simplex virus and are usually relatively easy to identify
[(['the', 'herpes', 'simplex', 'virus'], ['Cold', 'sores']), (['the', 'herpes', 'simplex', 'virus'], ['fever', 'blisters'])]
[(['the', 'herpes', 'simplex', 'virus'], ['Cold', 'sores']), (['the', 'herpes', 'simplex', 'virus'], ['fever', 'blisters'])]
------------------
Sentence-002 Fog , rain , darkness , and or blowing snow lead to disorientation
[(['Fog'], ['disorientation']), (['rain'], ['disorientation']), (['darkness'], ['disorientation']), (['blowing', 'snow'], ['disorientation'])]
[(['Fog'], ['disorientation']), (['rain'], ['disorientation']), (['darkness'], ['disorientation']), (['blowing', 'snow'], ['disorientation'])]
------------------
Sentence-003 Aware of the suffering caused by unmindful drinking , I am committed to cultivate good health , both physical and mental , for myself , my family , and my society
[(['unmindful', 'drinking'

------------------
Sentence-135 Traffic vibrations on the street outside had caused the movement of the light
[(['Traffic', 'vibrations'], ['the', 'movement'])]
[(['Traffic', 'vibrations'], ['the', 'movement'])]
------------------
Sentence-136 The cutaneous mycoses are caused by a homogeneous group of keratinophilic fungi termed the dermatophytes
[(['keratinophilic', 'fungi'], ['The', 'cutaneous', 'mycoses'])]
[(['a', 'homogeneous', 'group'], ['The', 'cutaneous', 'mycoses'])]
------------------
Sentence-137 Method according to claim 1 , characterized in that the time duration of the quality reduction is limited at least to the duration of the interference triggered by the switching process
[(['the', 'switching', 'process'], ['the', 'interference'])]
[(['the', 'switching', 'process'], ['the', 'interference'])]
------------------
Sentence-138 Smoke is one of the leading causes of kidney failures
[(['Smoke'], ['kidney', 'failures'])]
[(['Smoke'], ['kidney', 'failures'])]
-----------------

In [ ]:
predict('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-False_CRF/test/6_{198}.hdf5')

------------------
Sentence-001 Cold sores or fever blisters are caused by the herpes simplex virus and are usually relatively easy to identify
[(['the', 'herpes', 'simplex', 'virus'], ['Cold', 'sores']), (['the', 'herpes', 'simplex', 'virus'], ['fever', 'blisters'])]
[(['the', 'herpes', 'simplex', 'virus'], ['Cold', 'sores']), (['the', 'herpes', 'simplex', 'virus'], ['fever', 'blisters'])]
------------------
Sentence-002 Fog , rain , darkness , and or blowing snow lead to disorientation
[(['Fog'], ['disorientation']), (['rain'], ['disorientation']), (['darkness'], ['disorientation']), (['blowing', 'snow'], ['disorientation'])]
[(['Fog'], ['disorientation']), (['rain'], ['disorientation']), (['darkness'], ['disorientation']), (['blowing', 'snow'], ['disorientation'])]
------------------
Sentence-003 Aware of the suffering caused by unmindful drinking , I am committed to cultivate good health , both physical and mental , for myself , my family , and my society
[(['unmindful', 'drinking'

Sentence-162 The light in the background is from the sunrise
[(['the', 'sunrise'], ['The', 'light'])]
[(['the', 'sunrise'], ['The', 'light'])]
------------------
Sentence-163 We estimate a wind speed associated with the devastation caused by the tornado
[(['the', 'tornado'], ['the', 'devastation'])]
[(['the', 'tornado'], ['the', 'devastation'])]
------------------
Sentence-164 Heat , wind and smoke cause flight delays
[(['Heat'], ['flight', 'delays']), (['wind'], ['flight', 'delays']), (['smoke'], ['flight', 'delays'])]
[(['Heat'], ['flight', 'delays']), (['wind'], ['flight', 'delays']), (['smoke'], ['flight', 'delays'])]
------------------
Sentence-165 The elderly found it hard to cope with the high heat-humidity , which often causes nausea and dizziness from exhaustion and dehydration
[(['exhaustion'], ['nausea']), (['exhaustion'], ['dizziness']), (['dehydration'], ['nausea']), (['dehydration'], ['dizziness'])]
[(['exhaustion'], ['nausea']), (['exhaustion'], ['dizziness']), (['dehydr

### 2-002

In [ ]:
predict('your path to /FLAIR-True_CHAR-cnn_LSTM_MHSA-True_CRF/test/6_{96}.hdf5')

------------------
Sentence-001 Bounding pulses are caused by the relatively low systemic arterial blood pressure due to the continuous runoff of blood from the aorta into the pulmonary artery
[(['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'], ['Bounding', 'pulses']), (['the', 'continuous', 'runoff'], ['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'])]
[(['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'], ['Bounding', 'pulses']), (['the', 'continuous', 'runoff'], ['Bounding', 'pulses'])]
------------------
Sentence-002 This year's Nobel Laureates in Physiology or Medicine made the remarkable and unexpected discovery that inflammation in the stomach as well as ulceration of the stomach or duodenum is the result of an infection of the stomach caused by the bacterium Helicobacter pylori
[(['an', 'infection'], ['inflammation']), (['an', 'infection'], ['ulceration']), (['the', 'bacterium', 'Helicobacter', 'pylori'], ['

In [ ]:
predict('your path to /FLAIR-True_CHAR-None_LSTM_MHSA-False_CRF/test/6_{115}.hdf5')

------------------
Sentence-001 Bounding pulses are caused by the relatively low systemic arterial blood pressure due to the continuous runoff of blood from the aorta into the pulmonary artery
[(['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'], ['Bounding', 'pulses']), (['the', 'continuous', 'runoff'], ['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'])]
[(['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'], ['Bounding', 'pulses']), (['the', 'continuous', 'runoff'], ['Bounding', 'pulses'])]
------------------
Sentence-002 This year's Nobel Laureates in Physiology or Medicine made the remarkable and unexpected discovery that inflammation in the stomach as well as ulceration of the stomach or duodenum is the result of an infection of the stomach caused by the bacterium Helicobacter pylori
[(['an', 'infection'], ['inflammation']), (['an', 'infection'], ['ulceration']), (['the', 'bacterium', 'Helicobacter', 'pylori'], ['

In [ ]:
predict('your path to /FLAIR-False_CHAR-None_LSTM_MHSA-False_CRF/test/6_{198}.hdf5')

------------------
Sentence-001 Bounding pulses are caused by the relatively low systemic arterial blood pressure due to the continuous runoff of blood from the aorta into the pulmonary artery
[(['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'], ['Bounding', 'pulses']), (['the', 'continuous', 'runoff'], ['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'])]
[(['the', 'relatively', 'low', 'systemic', 'arterial', 'blood', 'pressure'], ['Bounding', 'pulses'])]
------------------
Sentence-002 This year's Nobel Laureates in Physiology or Medicine made the remarkable and unexpected discovery that inflammation in the stomach as well as ulceration of the stomach or duodenum is the result of an infection of the stomach caused by the bacterium Helicobacter pylori
[(['an', 'infection'], ['inflammation']), (['an', 'infection'], ['ulceration']), (['the', 'bacterium', 'Helicobacter', 'pylori'], ['an', 'infection'])]
[(['the', 'bacterium', 'Helicobacter', 